# Solving CartPole using Deep Q-Learning

Let's start by installing OpenAI Gym in order to get the CartPole environment

In [0]:
!pip install gym

Now to load all the packages we need

In [0]:
import os # FOR PYTHONHASHSEED
import gym
import numpy as np
import random
import tensorflow as tf
from collections import namedtuple
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


Setting the seeds in order to make our results reproducible

In [0]:
SEED = 9001

In [0]:
# 1. `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(SEED)


np.random.seed(SEED)
random.seed(SEED)


# Force TensorFlow to use single thread.
# Multiple threads are a potential source of non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)


# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)


## Experience Replay Class
This class allows for fast random memory access and holds only up to the max capacity

In [0]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'reward', 'next_state', 'done'))

In [0]:
# Implements a Ring Buffer
class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def append(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

## Deep Q-Learning Agent

The DQNAgent class is going to house all the logic necessary to have a Deep Q-Learning Agent. Since there's a lot going on here, this section will be longer than the others.

### Parameters

There are several parameters that are hard-coded into the model that should be tweaked when applying it to different problems to see if it affects performance. We will describe each parameter briefly here.



* Epsilon: The exploration rate. How often will the agent choose a random move during training instead of relying on information it already has. Helps the agent go down paths it normally wouldn't in hopes for higher long term rewards.
  - Epsilon Decay: How much our epsilon decreases after each update
  - Epsilon Min: The lowest rate of exploration we'll allow during training
* Gamma: Discount rate. This tells us how much we prioritize current versus future rewards.
* Tau: Affects how much we shift our knowledge based off of new information.
* Learning Rate: Affects the optimization procedures in the neural networks.



### Fixed Q-Targets

In Q-Learning we update our Q_Table through the following function:

$Q_{TableEntry}(state, action) = Reward + max(Q_{TableEntry}(state))$

Since our update is dependent on the same table itself, we can start to get correlated entries. This could cause oscillations to happen in our training. 

To combat this, we implemented a target model. It essentially is a copy of the original model, except that the values do not update as rapidly. The rate at which the target model updates is dependent upon `Tau` in our parameter list.

### Prioritized Replay
Create a probability vector giving priority to experiences with higher error

$ prob1 = error / sum(error) $

Then calculate the standard uniform probabilities
$ prob_unif = 1 / length(error) $

Finally, interpolate between the two using the hyperparameter $v$

$ prob = v * prob1 + (1 - v) * prob_unif $


### Agent Workflow

1. Create an empty neural network for both the model and target model.
2. Given a starting state, perform an action.
3. Once you performed the action on the environment, store the information gained into the agent's memory.
4. Once the agent acquires enough memory to meet the batch size requirements, select `batch_size` number of past memories randomly and train the model on those experiences.
5. The model will be trained by calculating the value of the current state and  updating the value proposed previously by the model.
6. The value of the state.
  - If the game is finished, then the reward is the value of that state.
  - If not, then take the current reward and add the discounted reward of future states.
7. Decay the epsilon value as described in the parameters.
8. Gradually update the target model.
9. Perform an action for the new state, either randomly through epsilon, or by choosing the best action based on what we currently know.
9. Repeat steps 4-9.

In [0]:
class DQNAgent:
  def __init__(self, state_size, action_size):
    self.state_size = state_size
    self.action_size = action_size
    # The deque will only contain the last 2000 entries
    self.memory = ReplayMemory(capacity=2000)
    self.v = 0.25 # How far away from uniform sampling? Max: 1
    self.gamma = 0.95 # Discount Rate
    self.epsilon = 1.0 # Exploration Rate
    self.epsilon_min = 0.001
    self.epsilon_decay = 0.9995
    self.learning_rate = 0.001
    self.model = self._build_model()
    ## Additional components for Fixed Q-Targets
    self.target_model = self._build_model()
    self.tau = 0.1 # We want to adjust our network by 10% each time
    
  def _build_model(self):
    model = Sequential()
    model.add(Dense(24, input_shape=(self.state_size,), activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(self.action_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
    return model
  
  def update_target_model(self):
    for layer_target, layer_src in zip(self.target_model.layers, self.model.layers):
      weights = layer_src.get_weights()
      target_weights = layer_target.get_weights()
      # Adjust the weights of the target to be tau proportion closer to the current
      for i in range(len(weights)):
            target_weights[i] = self.tau * weights[i] + (1 - self.tau) * target_weights[i]
      layer_target.set_weights(target_weights)
  
  def remember(self, state, action, reward, next_state, done):
    self.memory.append(state, action, reward, next_state, done)
    
  def act_random(self):
    return random.randrange(self.action_size)
  
  def best_act(self, state):
    # Choose the best action based on what we already know
    # If all the action values for a given state is the same, then act randomly
    action_values = self.model.predict(state)[0]
    action = self.act_random() if np.all(action_values[0] == action_values) else np.argmax(action_values)
    return action
  
  def act(self, state):
    action = self.act_random() if np.random.rand() <= self.epsilon else self.best_act(state)
    if self.epsilon > self.epsilon_min:
      self.epsilon *= self.epsilon_decay
    return action
  
  def discount_rewards(self, reward, next_state, done):
    return reward if done else reward + self.gamma * np.amax(self.target_model.predict(next_state)[0])
  
  def sample(self, batch_size):
    # If done do the other calculation
    td_error = np.array([abs(self.discount_rewards(reward, next_state, done) - self.target_model.predict(state)[0][action]) for state, action, reward, next_state, done in self.memory.memory])
    greedy_prob = td_error / np.sum(td_error)
    uniform_prob = np.full(greedy_prob.size, 1 / greedy_prob.size)
    prob = self.v * greedy_prob + (1 - self.v) * uniform_prob
    numpy_memory = np.array(self.memory.memory)
    return numpy_memory[np.random.choice(numpy_memory.shape[0], size = batch_size, replace = False, p = prob)].tolist()
  

  def replay(self, batch_size):
    minibatch = self.sample(batch_size)
    for state, action, reward, next_state, done in minibatch:
      target = reward
      if not done:
        target = reward + self.gamma * np.amax(self.target_model.predict(next_state)[0])
      action_values = self.model.predict(state)
      action_values[0][action] = target
      self.model.fit(state, action_values, epochs=1, verbose=0)
    
    self.update_target_model()

      
  
  def load(self, name):
    self.model.load_weights(name)
    
  def save(self, name):
    self.model.save_weights(name)

## Training

We will now use our Deep Q-Learning Agent to train it in CartPole by simulating a lot of runs through the environment.

In [0]:
%time
env = gym.make('CartPole-v1')
env.seed(SEED)
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

agent = DQNAgent(state_size, action_size)
done = False
batch_size = 32
EPISODES = 50

for episode_num in range(EPISODES):
  state = env.reset()
  state = np.reshape(state, [1, state_size])
  
  time = 0
  done = False
  while not done:
    action = agent.act(state)
    next_state, reward, done, _ = env.step(action)
    
    next_state = np.reshape(next_state, [1, state_size])
    
    agent.remember(state, action, reward, next_state, done)
    
    state = next_state
    
    if done:
      print("episode: {}/{}, score: {}, epsilon: {:.2}"
          .format(episode_num, EPISODES, time, agent.epsilon))
      break # We finished this episode
    
    if len(agent.memory) > batch_size:
      agent.replay(batch_size)
    
    time += 1
  

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.11 µs
episode: 0/50, score: 15, epsilon: 0.99
episode: 1/50, score: 16, epsilon: 0.98


/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


episode: 2/50, score: 19, epsilon: 0.97
episode: 3/50, score: 28, epsilon: 0.96
episode: 4/50, score: 30, epsilon: 0.95
episode: 5/50, score: 23, epsilon: 0.93
episode: 6/50, score: 15, epsilon: 0.93
episode: 7/50, score: 12, epsilon: 0.92
episode: 8/50, score: 11, epsilon: 0.91
episode: 9/50, score: 65, epsilon: 0.89
episode: 10/50, score: 20, epsilon: 0.88
episode: 11/50, score: 24, epsilon: 0.86
episode: 12/50, score: 49, epsilon: 0.84
episode: 13/50, score: 20, epsilon: 0.83
episode: 14/50, score: 10, epsilon: 0.83
episode: 15/50, score: 15, epsilon: 0.82
episode: 16/50, score: 23, epsilon: 0.81
episode: 17/50, score: 16, epsilon: 0.81
episode: 18/50, score: 27, epsilon: 0.8
episode: 19/50, score: 61, epsilon: 0.77
episode: 20/50, score: 76, epsilon: 0.74
episode: 21/50, score: 12, epsilon: 0.74
episode: 22/50, score: 49, epsilon: 0.72
episode: 23/50, score: 17, epsilon: 0.71
episode: 24/50, score: 19, epsilon: 0.71
episode: 25/50, score: 34, epsilon: 0.69
episode: 26/50, score: 38

## Evaluate Performance

Now to test how our agent performed, we will run through more scenarios except this time we don't allow the agent to choose randomly and have it rely on its previous experiences.

In [0]:
time_achieved = []
for episode_num in range(EPISODES):
  state = env.reset()
  state = np.reshape(state, [1, state_size])
  
  time = 0
  done = False
  while not done:
    action = agent.best_act(state)
    next_state, reward, done, _ = env.step(action)
    
    next_state = np.reshape(next_state, [1, state_size])
    
    
    state = next_state
    
    if done:
      print("episode: {}/{}, score: {}"
          .format(episode_num, EPISODES, time))
      time_achieved.append(time)
      break # We finished this episode

    time += 1
  

## Analysis of Performance

Let us load the pandas library to have some visualizations


In [0]:
import pandas as pd

In [0]:
time_achieved = pd.DataFrame(time_achieved, columns = ['time'])

Now to display a histogram of seconds that we lasted.

In [0]:
time_achieved.hist()

Summary Statistics for summarization

In [0]:
print("Mean: {} seconds\nStandard Deviation: {} seconds".format(time_achieved.time.mean(), time_achieved.time.std()))